In [1]:
import nltk
import re
import codecs
import csv
import os
from sklearn import metrics

path = 'D:\Dropbox\ongoing\chrisIntervention'

In [2]:
# read cohort info from the CSV and save them with the sbj ID

intvIdx = 'Ex3'

category = {}
with open(os.path.join(path, intvIdx + '_key.csv')) as f:
    sbjData = csv.reader(f)
    next(sbjData) # skip the header
    for row in sbjData:
        category[row[0]] = int(row[1])

subjectId = category.keys()        
        
#print(subjectId)
#print()
print('# of cohort 1: ', sum(1 for sbj in category if category[sbj] == 1))
print('# of cohort 2: ', sum(1 for sbj in category if category[sbj] == 2))


# of cohort 1:  76
# of cohort 2:  161


In [3]:
# build the dictionary based on the training set
# this is just to check if it is working
# we should build the dictionary for each leave-one-out test

def make_dictionary(subjectPool, critNumSbj):
    allWords = []
    for sbj in subjectPool:
        sbjWords = []
        for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
            line = line.strip()
            line = line.lower()
            tokens = nltk.wordpunct_tokenize(line)
            text = nltk.Text(tokens)
            words = [w for w in text if re.search('[a-z]+',w) and len(w)>2]
            sbjWords.extend(words)
        # when making dictionary, make the words from a document unique
        sbjWords = list(set(sbjWords))
        allWords.extend(sbjWords)
    
    # once we make list of all words, we count the words
    cntWords = nltk.FreqDist(allWords)
    #for ii in range(21,0,-2):
    #    print('# words with freq >', ii, ' : ', sum(1 for word in cntWords if cntWords[word] > ii))
    
    # we go with the words that came from at least N different people (arbitrary) 
    # we can change and see what happens ...
    return [word for word in cntWords if cntWords[word] > critNumSbj]

dictionary = make_dictionary(subjectId, 5)

print(dictionary)

['she', 'education', 'after', 'why', 'ball', 'stuff', 'care', 'could', 'out', 'second', 'year', 'who', 'work', 'live', 'get', 'sometimes', 'them', 'watch', 'own', 'through', 'her', 'these', 'homework', 'don', 'dad', 'about', 'has', 'world', 'games', 'what', 'bad', 'little', 'time', 'when', 'say', 'tell', 'and', 'outside', 'person', 'really', 'hang', 'feel', 'team', 'music', 'learn', 'only', 'people', 'two', 'will', 'since', 'real', 'everyday', 'being', 'you', 'years', 'listen', 'but', 'family', 'school', 'makes', 'were', 'every', 'humor', 'its', 'cheer', 'think', 'friend', 'listening', 'been', 'fun', 'not', 'some', 'something', 'enjoy', 'sense', 'any', 'myself', 'thing', 'always', 'spending', 'important', 'movies', 'your', 'back', 'their', 'how', 'all', 'laugh', 'better', 'everything', 'right', 'then', 'together', 'over', 'much', 'helps', 'than', 'sing', 'alot', 'even', 'gives', 'anything', 'spend', 'sad', 'with', 'take', 'brother', 'there', 'special', 'new', 'well', 'happy', 'another'

In [4]:
# we define a function that tunrs text into a list of tokens

def extract_feature(text, dictionary):
    text = text.strip()
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)
    feature = {}
    for word in dictionary:
        feature[word] = (word in text)
    return feature

print('nltk output: ', nltk.wordpunct_tokenize('You think you can do this one, Kate?'))
print(extract_feature('You think you can do this one, Kate?', dictionary))

nltk output:  ['You', 'think', 'you', 'can', 'do', 'this', 'one', ',', 'Kate', '?']
{'very': False, 'people': False, 'two': False, 'same': False, 'going': False, 'into': False, 'she': False, 'then': False, 'something': False, 'will': False, 'long': False, 'hard': False, 'education': False, 'what': False, 'after': False, 'listen': False, 'why': False, 'ball': False, 'football': False, 'bad': False, 'over': False, 'its': False, 'little': False, 'each': False, 'time': False, 'been': False, 'sister': False, 'basketball': False, 'know': False, 'mostly': False, 'any': False, 'since': False, 'friends': False, 'another': False, 'getting': False, 'from': False, 'real': False, 'when': False, 'everyday': False, 'our': False, 'just': False, 'being': False, 'help': False, 'someone': False, 'stay': False, 'video': False, 'was': False, 'sports': False, 'tell': False, 'thats': False, 'you': True, 'years': False, 'helps': False, 'calm': False, 'dance': False, 'sometimes': False, 'out': False, 'also': F

In [5]:
# leave-one-out classification

predLabel = []
trueLabel = []
currSbj = 1;

for sbj in subjectId:
    # sbj is being tested, others are used to train the classifier
    trainSbj = list(subjectId)
    trainSbj.remove(sbj)
    # create the dictionary
    trainDict = make_dictionary(trainSbj, 5)
    trainText = []
    # create the training set
    for tSbj in trainSbj:
        tmpText = ''
        for line in codecs.open(os.path.join(path, intvIdx, tSbj + '.txt'), 'r' , 'utf-8'):
            tmpText = tmpText + ' ' + line
        trainText.append([extract_feature(tmpText, trainDict), category[tSbj]])
    # train classifier
    classifier = nltk.NaiveBayesClassifier.train(trainText)

    # now the test set
    trueLabel.append( category[sbj] )
    testText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        testText = testText + ' ' + line
    # predict!
    currPred = classifier.classify(extract_feature(testText, trainDict))
    predLabel.append( currPred )
    
    print(str(currSbj) + ' (' + sbj + '): pred - ' + str(currPred) + ', actual - ' + str(category[sbj]) )
    currSbj = currSbj + 1

print('done. ')


1 (420022_3): pred - 2, actual - 2
2 (620103_3): pred - 2, actual - 2
3 (520025_3): pred - 2, actual - 2
4 (1120037_3): pred - 2, actual - 2
5 (820082_3): pred - 1, actual - 2
6 (720058_3): pred - 2, actual - 2
7 (510701_3): pred - 1, actual - 1
8 (610408_3): pred - 2, actual - 1
9 (620048_3): pred - 2, actual - 2
10 (810317_3): pred - 1, actual - 1
11 (820061_3): pred - 2, actual - 2
12 (720053_3): pred - 2, actual - 2
13 (1120100_3): pred - 2, actual - 2
14 (310076_3): pred - 2, actual - 1
15 (120039_3): pred - 2, actual - 2
16 (1120131_3): pred - 2, actual - 2
17 (110049_3): pred - 1, actual - 1
18 (310168_3): pred - 1, actual - 1
19 (1020133_3): pred - 2, actual - 2
20 (410401_3): pred - 2, actual - 1
21 (920119_3): pred - 2, actual - 2
22 (620100_3): pred - 2, actual - 2
23 (520061_3): pred - 2, actual - 2
24 (120022_3): pred - 2, actual - 2
25 (810903_3): pred - 2, actual - 1
26 (1020098_3): pred - 2, actual - 2
27 (420037_3): pred - 2, actual - 2
28 (110009_3): pred - 2, actual 

In [6]:
# summarize the results

print(metrics.classification_report(trueLabel, predLabel))


             precision    recall  f1-score   support

          1       0.83      0.57      0.67        76
          2       0.82      0.94      0.88       161

avg / total       0.82      0.82      0.81       237



In [11]:
# ok, it is POSSIBLE TO PREDICT cohort from the writings.
# now, show me the informative features

dictionary = make_dictionary(subjectId, 5)
print(len(dictionary))

allText = []

for sbj in subjectId:
    tmpText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        tmpText = tmpText + ' ' + line
        allText.append([extract_feature(tmpText, dictionary), category[sbj]])

# train classifier based on all information
classifier = nltk.NaiveBayesClassifier.train(allText)


212


In [12]:
classifier.show_most_informative_features(100)


Most Informative Features
                     had = True                1 : 2      =     10.5 : 1.0
                    long = True                1 : 2      =     10.5 : 1.0
                   watch = True                1 : 2      =     10.5 : 1.0
                 outside = True                1 : 2      =      9.1 : 1.0
                   house = True                1 : 2      =      7.7 : 1.0
                   learn = True                1 : 2      =      7.7 : 1.0
                   years = True                1 : 2      =      7.7 : 1.0
                     own = True                1 : 2      =      7.7 : 1.0
                   those = True                1 : 2      =      7.7 : 1.0
                   games = True                1 : 2      =      7.5 : 1.0
                    ball = True                1 : 2      =      7.2 : 1.0
                     now = True                1 : 2      =      6.3 : 1.0
                    this = True                1 : 2      =      6.3 : 1.0